In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import math
import random

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import set_random_seed
from tensorflow.python.client import device_lib
import datetime as dt
import os

Using TensorFlow backend.


In [3]:
tf.__version__

'1.13.1'

In [4]:
saved_model_dir = "saved_model/general_full_data_training/"
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
tf.reset_default_graph()

saving_dir = "explain/general_full_data_training/"
if not os.path.exists(saving_dir):
    os.makedirs(saving_dir)

# fix random seed for reproducibility
np.random.seed(7)
set_random_seed(7)

NUM_LAYERS = 1
HIDDEN_SIZE = 64
TRAIN_EPOCHS = 300

print_iter = 50

TRAIN_NUM_STEPS = 4
TRAIN_BATCHES = 2048
TEST_SIZE = 0

seq_length = 4
# with open('pd_list_full_with_rewards_original_seq4.pkl', 'rb') as f:
#     pd_list = pickle.load(f)

with open('huge_pd_shuffled_with_rewards_original_seq4_SHUFFLED.pkl', 'rb') as f:
    huge_pd = pickle.load(f)

In [5]:
def my_train_test_split(X, y, sequence_length, test_size=TEST_SIZE, debug_print=False):
    train_samples = int(np.ceil(X.shape[0] * (1 - test_size)))
    test_samples = int(np.floor(X.shape[0] * test_size))
    if debug_print: print("train samples:", train_samples, " test samples:", test_samples)
    divide_train = train_samples % sequence_length == 0
    while not (divide_train):
        train_samples += 1
        divide_train = train_samples % sequence_length == 0

    divide_test = test_samples % sequence_length == 0
    while not (divide_test):
        test_samples -= 1
        divide_test = test_samples % sequence_length == 0

    print("train can divide by ", sequence_length, "?", divide_train, "num train samples = ", train_samples)
    print("test can divide by ", sequence_length, "?", divide_test, "num train samples = ", test_samples)
    X_train = X[0:train_samples].copy()
    if debug_print: print("X_train indices are 0 to:", train_samples)
    X_test = X[train_samples:].copy()
    if debug_print: print("X_test indices are {} to {}:".format(train_samples, len(X)))
    y_train = y[0:train_samples].copy()
    if debug_print: print("y_train indices are {} to {}:".format(0, train_samples))
    y_test = y[train_samples:].copy()
    if debug_print: print("y_test indices are {} to {}:".format(train_samples, len(y)))
    # TODO: enter assertion to check sizes X_train/test and the samples computed earlier
    # TODO: assert of train size = labels etc
    return X_train, X_test, y_train, y_test

In [7]:
cur_data = huge_pd.copy()

cur_data['choice'] = cur_data.choice.apply(lambda x: x - 1)
cur_data['prev_choice'] = cur_data.prev_choice.apply(lambda x: x - 1)

# X = cur_data.drop(columns = ['orig_choice_num','index','choice','user','time','reward','payoff_structure','reward_1','reward_2','reward_3','reward_4'])
X = cur_data.drop(
    columns=['index', 'choice', 'user', 'time', 'reward', 'payoff_structure', 'reward_1', 'reward_2', 'reward_3',
             'reward_4'])
X_prev = to_categorical(X.prev_choice, dtype='int64')
y = cur_data.choice
num_of_classes = len(y.unique())
y = to_categorical(y, dtype='int64')

new_X = []
for prev_choice, prev_reward in zip(X_prev, X.prev_reward):
    new_i = np.append(prev_choice, prev_reward)
    new_X.append(new_i)
new_X = np.array(new_X)

seq_data = cur_data.reset_index(drop=True).copy()

In [39]:
def is_continous_sequence(choice_numbers):
# gets a list and return True if the sequence is continuous without gaps, otherwise return False
    for c, choice_number in enumerate(choice_numbers):
        if c==1:
            continue
        if choice_numbers[c] - choice_numbers[c-1] > 1:
            return False
    return True

In [61]:
start = 0
end = 4

continuous_counter = 0
not_continuous_counter = 0

more_than_one_gap_seq = []
for i in range(start, seq_data.shape[0],4):
    if i == 0 :
        start = 0
        end = start + 4
    start = i
    end = start + 4
    cur_orig_choices = list(seq_data[start:end]['orig_choice_num'])
    if is_continous_sequence(cur_orig_choices):
        continuous_counter += 1
    else:
        not_continuous_counter += 1
        more_than_one_gap_seq.append(cur_orig_choices)

In [75]:
seq_data[start:end]

index  user  choice  reward    time  payoff_structure  reward_1  \
543820    144   159       0      44  1061.0                 2        44   
543821    145   159       1      35   956.0                 2        47   
543822    146   159       2      61   908.0                 2        46   
543823    147   159       0      46   915.0                 2        46   

        reward_2  reward_3  reward_4  orig_choice_num  prev_choice  \
543820        38        60        55              144            3   
543821        35        62        48              145            0   
543822        47        61        57              146            1   
543823        35        70        43              147            2   

        prev_reward  
543820           51  
543821           44  
543822           35  
543823           61

In [76]:
seq_data.shape[0]//4

135956

In [62]:
assert(continuous_counter+not_continuous_counter==seq_data.shape[0]//4)

In [63]:
continuous_counter

130779

In [64]:
not_continuous_counter

5177

In [66]:
def how_continous_sequence(choice_numbers):
# gets a list and returns : the highest gap in the sequence, total gap
    highest_gap = 0
    gap_sum = 0
    for c, choice_number in enumerate(choice_numbers):
        if c==1:
            continue
        gap = choice_numbers[c] - choice_numbers[c-1] 
        if gap>highest_gap:
            highest_gap = gap
        gap_sum += gap
        
    return highest_gap, gap_sum

In [67]:
# check the more_than_one_gap_seq - how many gaps there is 
highest_gaps = []
gaps_sums = []
for i in range(len(more_than_one_gap_seq)):
    highest_gap, gap_sum = how_continous_sequence(more_than_one_gap_seq[i])
    highest_gaps.append(highest_gap)
    gaps_sums.append(gap_sum)

In [70]:
from collections import Counter

In [72]:
highest_gaps_counter = Counter(highest_gaps)
gaps_sums_counter = Counter(gaps_sums)

In [73]:
highest_gaps_counter

Counter({2: 4569,
         3: 384,
         4: 105,
         10: 6,
         7: 24,
         5: 32,
         9: 4,
         76: 2,
         15: 2,
         6: 22,
         21: 2,
         17: 1,
         8: 5,
         13: 2,
         14: 4,
         27: 2,
         34: 1,
         39: 2,
         11: 6,
         22: 2})

---------

In [7]:
X_train, X_test, y_train, y_test = my_train_test_split(new_X, y, seq_length, test_size=TEST_SIZE, debug_print=True)
# X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size = 0.2, random_state = 42,shuffle=False)

possible_samples = int(X.shape[0] / seq_length)
possible_samples_train = int(X_train.shape[0] / seq_length)
possible_samples_test = int(X_test.shape[0] / seq_length)
possible_labels_train = int(y_train.shape[0] / seq_length)
possible_labels_test = int(y_test.shape[0] / seq_length)

# reshape X to be [samples, time steps, features]
X_train = np.reshape(X_train, (possible_samples_train, seq_length, X_train.shape[1]))

# y_cat_train = np.reshape(y_train, (seq_length, possible_samples_train, y_train.shape[1]))    # coursera's shape
y_cat_train = np.reshape(y_train, (possible_samples_train, seq_length, y_train.shape[1]))

X_test = np.reshape(X_test, (possible_samples_test, seq_length, X_test.shape[1]))
y_cat_test = np.reshape(y_test, (possible_labels_test, seq_length, y_test.shape[1]))

num_of_samples = y_cat_test.shape[0]

train samples: 543824  test samples: 0
train can divide by  4 ? True num train samples =  543824
test can divide by  4 ? True num train samples =  0
X_train indices are 0 to: 543824
X_test indices are 543824 to 543824:
y_train indices are 0 to 543824:
y_test indices are 543824 to 543824:


In [8]:
X_train.shape

(135956, 4, 5)

In [9]:
class Model(object):
    def __init__(self,all_x,all_y, is_training, output_size, dropout=1.0, batch_size=TRAIN_BATCHES, return_seqence=False):

        # self.x = tf.placeholder(dtype=tf.int32, shape=[None, 4, 5], name='X_placeholder')
        # self.y = tf.placeholder(dtype=tf.int32, shape=[None, 4], name='Y_placeholder')

        # A dataset from a tensor
        dataset = tf.data.Dataset.from_tensor_slices(all_x)
        # Divide the dataset into batches. Once you reach the last batch which won't be 512, the dataset will know exactly which elements remain and should be passed as a batch.
        dataset = dataset.batch(TRAIN_BATCHES)
        # An iterator that can be reinitialized over and over again, therefore having a new shuffle of the data each time
        self.iterator = dataset.make_initializable_iterator()
        # A node that can be run to obtain the next element in the dataset. However, this node will be linked in the model so obtaining the next element will be done automatically
        self.data_X = self.iterator.get_next()

        labels = tf.data.Dataset.from_tensor_slices(all_y)
        # Shuffle the dataset with some arbitrary buffer size
        # dataset = dataset.shuffle(buffer_size=10)
        # Divide the dataset into batches. Once you reach the last batch which won't be 512, the dataset will know exactly which elements remain and should be passed as a batch.
        labels = labels.batch(TRAIN_BATCHES)
        # An iterator that can be reinitialized over and over again, therefore having a new shuffle of the data each time
        self.labels_iterator = labels.make_initializable_iterator()
        # A node that can be run to obtain the next element in the dataset. However, this node will be linked in the model so obtaining the next element will be done automatically
        self.data_Y = self.labels_iterator.get_next()



        self.seq_len = tf.placeholder(dtype=tf.int32,name='sequence_len')


        cell = tf.nn.rnn_cell.LSTMCell(HIDDEN_SIZE)
        self.current_batch_size = tf.shape(self.data_X)[0]
        init_state = cell.zero_state(self.current_batch_size, tf.float32)
        self.output, self.states = tf.nn.dynamic_rnn(cell=cell, inputs=tf.cast(self.data_X, tf.float32), initial_state=init_state)

        # tf.keras.layers.Dense(output_size, activation=tf.nn.softmax)

        # reshape to (batch_size * num_steps, HIDDEN_SIZE)
        output = tf.reshape(self.output, [-1, HIDDEN_SIZE])

        softmax_w = tf.Variable(tf.random_uniform([HIDDEN_SIZE, output_size]))
        softmax_b = tf.Variable(tf.random_uniform([output_size]))

        self.logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
        # Reshape logits to be a 3-D tensor for sequence loss
        # if return_seqence:
        self.logits_reshaped = tf.reshape(self.logits, [self.current_batch_size, self.seq_len, output_size])[:,-1,:]

        # TODO: I need return_sequence false , which means I only needs the last output/hidden state
        self.softmax_out = tf.nn.softmax(self.logits)
        self.softmax_out_reshaped = tf.reshape(self.softmax_out, [self.current_batch_size,seq_length,output_size])[:,-1,:]
        #         weight_mask = tf.to_float(tf.sequence_mask([self.batch_size, self.num_steps],num_steps*batch_size))
        # Use the contrib sequence loss and average over the batches
        # loss = tf.contrib.seq2seq.sequence_loss(
        #     self.logits_reshaped,
        #     self.data_Y,
        #     tf.ones([self.current_batch_size, self.seq_len], dtype=tf.float32),
        #     average_across_timesteps=True,
        #     average_across_batch=False)
        # loss = tf.nn.softmax_cross_entropy_with_logits_v2(tf.cast(self.data_Y,tf.float32), self.logits_reshaped)


        loss = tf.keras.losses.categorical_crossentropy(tf.cast(self.data_Y,tf.float32), self.logits_reshaped,from_logits=True)


        self.cost = tf.reduce_sum(loss)

        # get the prediction accuracy
        self.predict = tf.cast(tf.argmax(self.softmax_out, axis=1), tf.int64)
        self.predict_return_sequence_false = tf.cast(tf.argmax(self.softmax_out_reshaped, axis=1), tf.int64)

        # self.correct_prediction = tf.equal(tf.argmax(self.predict , 1), tf.argmax(self.data_Y, 1))
        self.correct_prediction = tf.equal(self.predict, tf.reshape(self.data_Y, [-1]))
        self.correct_prediction_return_sequence_false = tf.equal(self.predict_return_sequence_false, tf.argmax(self.data_Y, 1))

        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        self.accuracy_return_sequence_false = tf.reduce_mean(tf.cast(self.correct_prediction_return_sequence_false, tf.float32))
        # tf.keras.metrics.categorical_accuracy(y_true,y_pred)

        if not is_training:
            return

        # keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

        # tvars = tf.trainable_variables()
        # grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), 5)
        # #         optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
        # optimizer = tf.train.AdamOptimizer(0.001)
        # self.train_op = optimizer.apply_gradients(
        #     zip(grads, tvars),
        #     global_step=tf.contrib.framework.get_or_create_global_step())

        # self.optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.9999).minimize(self.cost)


In [10]:
class SinglesModel(object):
    def __init__(self,is_training, output_size, dropout=1.0, batch_size=TRAIN_BATCHES, return_seqence=False, single_test=False):

        self.data_X = tf.placeholder(dtype=tf.int64, shape=[1, 4, 5], name='X_placeholder')
        self.data_Y = tf.placeholder(dtype=tf.int64, shape=[1, 4], name='Y_placeholder')
        
        self.seq_len = tf.placeholder(dtype=tf.int32,name='sequence_len')


        cell = tf.nn.rnn_cell.LSTMCell(HIDDEN_SIZE)
        self.current_batch_size = tf.shape(self.data_X)[0]
        init_state = cell.zero_state(self.current_batch_size, tf.float32)
        self.output, self.states = tf.nn.dynamic_rnn(cell=cell, inputs=tf.cast(self.data_X, tf.float32), initial_state=init_state)

        output = tf.reshape(self.output, [-1, HIDDEN_SIZE])

        softmax_w = tf.Variable(tf.random_uniform([HIDDEN_SIZE, output_size]))
        softmax_b = tf.Variable(tf.random_uniform([output_size]))

        self.logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
        # Reshape logits to be a 3-D tensor for sequence loss
        # if return_seqence:
        self.logits_reshaped = tf.reshape(self.logits, [self.current_batch_size, self.seq_len, output_size])[:,-1,:]

        # TODO: I need return_sequence false , which means I only needs the last output/hidden state
        self.softmax_out = tf.nn.softmax(self.logits)
        self.softmax_out_reshaped = tf.reshape(self.softmax_out, [self.current_batch_size,seq_length,output_size])[:,-1,:]


        loss = tf.keras.losses.categorical_crossentropy(tf.cast(self.data_Y,tf.float32), self.logits_reshaped,from_logits=True)


        self.cost = tf.reduce_sum(loss)


        # get the prediction accuracy
        self.predict = tf.cast(tf.argmax(self.softmax_out, axis=1), tf.int64)
        self.predict_return_sequence_false = tf.cast(tf.argmax(self.softmax_out_reshaped, axis=1), tf.int64)

        # self.correct_prediction = tf.equal(tf.argmax(self.predict , 1), tf.argmax(self.data_Y, 1))
        self.correct_prediction = tf.equal(self.predict, tf.reshape(self.data_Y, [-1]))
        self.correct_prediction_return_sequence_false = tf.equal(self.predict_return_sequence_false, tf.argmax(self.data_Y, 1))

        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        self.accuracy_return_sequence_false = tf.reduce_mean(tf.cast(self.correct_prediction_return_sequence_false, tf.float32))
        # tf.keras.metrics.categorical_accuracy(y_true,y_pred)

        if not is_training:
            return
        # self.optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.9999).minimize(self.cost)
    
    def test_single(self,all_x, all_y, model_path, output_size=4, print_iter=50, print_results=True):
        # setup data and models
    #     Model(all_x,all_y,is_training=False, output_size=4, batch_size=TRAIN_BATCHES, test_single=True)
#         m = model
        init_op = tf.global_variables_initializer()
        saver = tf.train.Saver()

        with tf.Session() as sess:
            sess.run([init_op])
            saver.restore(sess, model_path)
            counter = 0
            # print("will run for {} steps in total".format(len(all_x)))
            accs_over_batches = []
            loss_over_batches = []
            predictions_over_batches = []
            correct_predictions = []
            hidden_states = []
            logits_list = []
            softmaxes = []
            outputs = []
            output, states, \
            logits, logits_reshaped, softmax_out, softmax_out_reshaped,\
            predict, predict_return_sequence_false, correct_prediction, correct_prediction_return_sequence_false,\
            accuracy, accuracy_return_sequence_false,\
                cost = sess.run(
                [
                 self.output, self.states,
                 self.logits, self.logits_reshaped, self.softmax_out, self.softmax_out_reshaped,
                 self.predict, self.predict_return_sequence_false, self.correct_prediction, self.correct_prediction_return_sequence_false,
                 self.accuracy, self.accuracy_return_sequence_false,
                 self.cost],
                feed_dict={
                           self.seq_len: TRAIN_NUM_STEPS, self.data_X:all_x, self.data_Y:all_y
                           })
            # print()
            counter = counter + 1
            accs_over_batches.append(accuracy_return_sequence_false)
            loss_over_batches.append(cost)
            predictions_over_batches.append(predict_return_sequence_false)
            correct_predictions.append(correct_prediction_return_sequence_false)
            hidden_states.append(states)
            logits_list.append(logits_reshaped)
            softmaxes.append(softmax_out_reshaped)
            outputs.append(output)

            if print_results:
                print("---------------------------------------------------------------")
                # print(accs_over_batches[-1])
                print(np.mean(accs_over_batches))
                print("loss:")
                # print(loss_over_batches[-1])
                print(np.mean(loss_over_batches))
            # do a final save
            # saver.save(sess, save_path)

            return accs_over_batches, loss_over_batches, predictions_over_batches, correct_predictions, hidden_states, logits_list, softmaxes, outputs


In [11]:
def train(all_x, all_y, output_size=4, print_iter=50):
    # setup data and models
    m = Model(all_x,all_y,is_training=True, output_size=4, batch_size=TRAIN_BATCHES)
    init_op = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run([init_op])
        saver = tf.train.Saver()
        accs_over_epochs = []
        loss_over_epochs = []
        counter = 0
        # print("will run for {} steps in total".format(len(all_x)))
        for epoch in range(TRAIN_EPOCHS):
            accs_over_batches = []
            loss_over_batches = []
            sess.run([m.iterator.initializer, m.labels_iterator.initializer])
            try:
                # As long as there are elements execute the block below
                while True:
                    """
                    xs and ys are the current batches
                    output and states are the returned values of tf.nn.dynamic_rnn
                    """
                    xs,ys, \
                    output, states, \
                    logits, logits_reshaped, softmax_out, softmax_out_reshaped,\
                    predict, predict_return_sequence_false, correct_prediction, correct_prediction_return_sequence_false,\
                    accuracy, accuracy_return_sequence_false,\
                        cost, _= sess.run(
                        [m.data_X,m.data_Y,
                         m.output, m.states,
                         m.logits, m.logits_reshaped, m.softmax_out, m.softmax_out_reshaped,
                         m.predict, m.predict_return_sequence_false, m.correct_prediction, m.correct_prediction_return_sequence_false,
                         m.accuracy, m.accuracy_return_sequence_false,
                         m.cost,
                         m.optimizer],
                        feed_dict={
                                   m.seq_len: TRAIN_NUM_STEPS
                                   })
                    # print()
                    counter = counter +1
                    accs_over_batches.append(accuracy_return_sequence_false)
                    loss_over_batches.append(cost)
            except tf.errors.OutOfRangeError:
                print("finished epoc {}, acc {} loss {}".format(epoch, np.mean(accs_over_batches),np.mean(loss_over_batches)))
                accs_over_epochs.append(np.mean(accs_over_batches))
                loss_over_epochs.append(np.mean(loss_over_batches))
                print(accs_over_batches)
                pass
        print("---------------------------------------------------------------")
        print(accs_over_epochs[-1])
        print(np.mean(accs_over_epochs))
        print("loss:")
        print(loss_over_epochs[-1])
        print(np.mean(loss_over_epochs))
        # do a final save
        saver.save(sess, save_path)

        return accs_over_epochs, loss_over_epochs


In [12]:
def test(all_x, all_y, model_path, output_size=4, print_iter=50, print_results=True):
    # setup data and models
    m = Model(all_x,all_y,is_training=False, output_size=4, batch_size=TRAIN_BATCHES)
    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run([init_op])
        saver.restore(sess, model_path)
        counter = 0
        # print("will run for {} steps in total".format(len(all_x)))
        accs_over_batches = []
        loss_over_batches = []
        predictions_over_batches = []
        correct_predictions = []
        hidden_states = []
        logits_list = []
        softmaxes = []
        outputs = []
        sess.run([m.iterator.initializer, m.labels_iterator.initializer])
        try:
            # As long as there are elements execute the block below
            while True:
                """
                xs and ys are the current batches
                output and states are the returned values of tf.nn.dynamic_rnn
                """
                xs,ys, \
                output, states, \
                logits, logits_reshaped, softmax_out, softmax_out_reshaped,\
                predict, predict_return_sequence_false, correct_prediction, correct_prediction_return_sequence_false,\
                accuracy, accuracy_return_sequence_false,\
                    cost = sess.run(
                    [m.data_X,m.data_Y,
                     m.output, m.states,
                     m.logits, m.logits_reshaped, m.softmax_out, m.softmax_out_reshaped,
                     m.predict, m.predict_return_sequence_false, m.correct_prediction, m.correct_prediction_return_sequence_false,
                     m.accuracy, m.accuracy_return_sequence_false,
                     m.cost],
                    feed_dict={
                               m.seq_len: TRAIN_NUM_STEPS
                               })
                # print()
                counter = counter + 1
                accs_over_batches.append(accuracy_return_sequence_false)
                loss_over_batches.append(cost)
                predictions_over_batches.append(predict_return_sequence_false)
                correct_predictions.append(correct_prediction_return_sequence_false)
                hidden_states.append(states)
                logits_list.append(logits_reshaped)
                softmaxes.append(softmax_out_reshaped)
                outputs.append(output)
        except tf.errors.OutOfRangeError:
            if print_results:
                print("finished testing, acc {} loss {}".format(np.mean(accs_over_batches),np.mean(loss_over_batches)))
                print(accs_over_batches)
            pass
        if print_results:
            print("---------------------------------------------------------------")
            # print(accs_over_batches[-1])
            print(np.mean(accs_over_batches))
            print("loss:")
            # print(loss_over_batches[-1])
            print(np.mean(loss_over_batches))
        # do a final save
        # saver.save(sess, save_path)

        return accs_over_batches, loss_over_batches, predictions_over_batches, correct_predictions, hidden_states, logits_list, softmaxes, outputs


In [15]:
### TRAINING PART ####

# accuracies_per_epoch, losses_per_epoch = train(X_train, y_cat_train[:,-1,:], save_path)
# train_results = accuracies_per_epoch, losses_per_epoch
# with open(os.path.join(saved_model_dir, 'model_with_keras_batch{}_results.pkl'.format(TRAIN_BATCHES)), 'wb') as handle:
#     pickle.dump(train_results, handle)

# tf.reset_default_graph()
# accuracies_test, losses_test, predictions_test, correct_predictions_test, hidden_states, logits_test, softmax_test, outputs_test = test(X_test, y_cat_test[:,-1,:],save_path)


## simulating with custom inputs

In [14]:
def get_last_seq(person_df, choice_number, seq_len, debug_prints=False):
    """
    Gets a person's data and a choice number and returns a sequence of previous 4 (or seq_length) choices made (maybe not 4 (seq_length) consecutives)
    """
    if choice_number < seq_len:
        if debug_prints: print("must be {} and more to get a sequence".format(seq_len))
        return -2
    ind = np.where(person_df["orig_choice_num"] == choice_number)
    if len(ind[0]) == 0:
        if debug_prints: print("choice number {} not found".format(choice_number))
        return -9
    elif len(ind[0]) > 1:
        if debug_prints: print("FOR SOME REASON THERE'S MORE THAN ONE CHOICE NUMBER LIKE {}".format(choice_number))
        return -1
    # got here so I found the choice number , now lets check if we have 4 previous choices
    seq = []
    for i in range(ind[0][0] - seq_len + 1, ind[0][0] + 1, 1):
        seq.append(i)

    return person_df.iloc[seq]

In [15]:
def create_data_sequence(seq_df, seq_len):
    """
    gets a SINGLE sequence in df form and returns it in a train/test ready form (numpy) X and y
    """
    cur_data = seq_df.copy()
    cur_data['choice'] = cur_data.choice.apply(lambda x: x - 1)
    cur_data['prev_choice'] = cur_data.prev_choice.apply(lambda x: x - 1)
    X = cur_data.drop(
        columns=['choice', 'user', 'time', 'reward', 'payoff_structure', 'reward_1', 'reward_2', 'reward_3', 'reward_4',
                 'orig_choice_num'])
    X_prev = to_categorical(X.prev_choice, num_classes=4, dtype='int64')
    y = cur_data.choice
    num_of_classes = len(y.unique())
    y = to_categorical(y, num_classes=4, dtype='int64')

    new_X = []
    for prev_choice, prev_reward in zip(X_prev, X.prev_reward):
        new_i = np.append(prev_choice, prev_reward)
        new_X.append(new_i)
    new_X = np.array(new_X)

    # reshape X to be [samples, time steps, features]
    X_reshaped = np.reshape(new_X, (1, seq_length, new_X.shape[1]))
    y_reshaped = np.reshape(y, (1, seq_length, y.shape[1]))

    return X_reshaped, y_reshaped

In [16]:
def create_fake_sequence(seq, rewards, y=1):
    """
    seq - is the input : array with 4 ints in range 0-3
    rewards - are the corresponding rewards array at each time
    returns x, y in a sequence format
    """
    
    X_prev = to_categorical(seq, num_classes=4, dtype='int64')
    if y<0:
        y = random.randint(1, 4)
    y = to_categorical(y, num_classes=4, dtype='int64')

    new_X = []
    for prev_choice, prev_reward in zip(X_prev, rewards):
        new_i = np.append(prev_choice, prev_reward)
        new_X.append(new_i)
    new_X = np.array(new_X)

    # reshape X to be [samples, time steps, features]
    X_reshaped = np.reshape(new_X, (1, seq_length, new_X.shape[1]))
    y_reshaped = np.reshape(y, (1, seq_length))

    return X_reshaped, y_reshaped

In [17]:
def make_prediction(input_seq, input_rewards, print_to_screen=True, print_to_log=True):
    """
    input: a sequence and rewards , both as lists of ints
    returns: a prediction
    """
    fake_x, fake_y = create_fake_sequence(seq=input_seq,rewards=input_rewards)
    tf.reset_default_graph()
    model_fake = SinglesModel(is_training=False, output_size=4, batch_size=TRAIN_BATCHES, single_test=True)
    current_acc, current_loss, class_pred, correct_prediction, hidden_states, logits_test, softmax_test, outputs_test = model_fake.test_single(fake_x, fake_y, save_path, print_results=False)
    if print_to_screen:
        print("predicted class: ",class_pred[0])
        print(softmax_test)
        
    return (class_pred[0],softmax_test)

In [18]:
def run_fake_seq():
    """
    wrapper for running a new sequence
    if input is "exit" at any phase, exiting the outer loop
    """
    g = input("Enter a sequence of choices: ")
    if g == "exit":
        return 1
    input_seq = [int(x) for x in g.split()]
    r = input("Enter the corresponding rewards: ")
    if g == "exit":
        return 1
    input_rewards = [int(x) for x in r.split()]

    make_prediction(input_seq, input_rewards)
    
    return 0

In [19]:
def continuous_run():
    halt = 0
    while not halt:
        halt = run_fake_seq()

In [20]:
def split_prediction_argmax(cell):
    splitted_cell = cell.split()
    return int(splitted_cell[0]), float(splitted_cell[1].replace('[','').replace(']',''))

In [21]:
def produce_comparison_df(output_with_argmax, generated_output_with_argmax):
    predict_compare = []
    argmax_compare = []
    for gen_row, orig_row in zip(generated_output_with_argmax.to_numpy(), output_with_argmax.to_numpy()):
        row_predictions_compare = []
        row_argmax_compare = []
        for gen_out, orig_out in zip(gen_row, orig_row):
            gen_pred, gen_argmax = split_prediction_argmax(gen_out)
            orig_pred, orig_argmax = split_prediction_argmax(orig_out)
            row_predictions_compare.append(int(gen_pred==orig_pred))
            row_argmax_compare.append(np.abs(gen_argmax-orig_argmax))
        predict_compare.append(row_predictions_compare)
        argmax_compare.append(row_argmax_compare)
    
    predict_compare_df = pd.DataFrame(np.asmatrix(predict_compare), index=generated_output_with_argmax.index, columns=generated_output_with_argmax.columns)
    argmax_compare_df = pd.DataFrame(np.asmatrix(argmax_compare), index=generated_output_with_argmax.index, columns=generated_output_with_argmax.columns)
    return predict_compare_df, argmax_compare_df

In [22]:
################## V5.1 FULL #############################
# Patterns:
constant_group = ['1 1 1 1', '2 2 2 2', '3 3 3 3', '4 4 4 4']
one_different = ['2 2 2 1', '4 2 2 2', '2 2 2 4', '1 2 2 2', '3 2 2 2', '2 3 3 3', '2 1 1 1']
repeating_two = ['1 2 1 2', '2 1 2 1', '2 3 2 3', '3 2 3 2', '4 3 4 3', '3 4 3 4']
all_different = ['1 2 3 4', '4 3 2 1', '2 3 4 1', '1 4 3 2', '3 4 1 2', '2 1 4 3', '4 1 2 3', '3 2 1 4']

all_patterns_groups = [constant_group, one_different, repeating_two, all_different]


# Rewards:
constant_group_rew = ['10 10 10 10', '15 15 15 15', '20 20 20 20', '25 25 25 25', '30 30 30 30', '35 35 35 35',
                      '40 40 40 40', '45 45 45 45', '50 50 50 50', '55 55 55 55', '60 60 60 60', '65 65 65 65',
                      '70 70 70 70', '75 75 75 75', '80 80 80 80', '85 85 85 85', '90 90 90 90']

ascending_rew = ['10 20 30 40', '15 20 25 30', '10 30 50 70', '10 40 70 90', '30 35 40 45', '45 50 55 65',
                 '50 55 65 75', '55 65 75 85', '65 75 85 95', '75 80 85 90', '10 15 20 25', '25 45 65 85',
                 '40 60 80 90', '60 70 80 90', '20 30 70 80', '40 50 60 70', '50 60 70 80']

descending_rew = ['90 80 70 60', '80 70 60 50', '70 60 50 40', '60 50 40 30', '50 40 30 20', '40 30 20 10',
                  '90 70 50 30', '70 50 30 10', '95 75 55 35', '75 55 35 15', '80 60 40 20', '85 65 45 25',
                  '90 70 40 10', '90 60 30 10', '90 50 30 10', '90 40 30 20', '90 30 20 10', '60 30 20 10']


one_different_rew_good = ['10 10 10 90', '20 20 20 90', '30 30 30 90', '40 40 40 90', '50 50 50 90', '60 60 60 90', 
                          '10 10 90 10', '20 20 90 20', '30 30 90 30', '40 40 90 40', '50 50 90 50', '20 80 20 20',
                          '30 90 30 30', '60 90 60 60', '80 20 20 20', '90 30 30 30', '90 40 40 40', '90 50 50 50',
                          '20 20 20 80', '40 40 40 80', '60 60 60 80', '20 20 20 40', '40 40 40 60', '20 20 20 60',
                          '20 20 80 20', '40 40 80 40', '60 60 80 40', '20 20 40 20', '40 40 60 40', '20 20 60 20',
                          '20 80 20 20', '40 80 40 40', '60 80 60 60', '20 40 20 20', '40 60 40 40', '20 60 20 20',
                          '80 20 20 20', '80 40 40 40', '80 60 60 60', '40 20 20 20', '60 40 40 40', '60 20 20 20']


one_different_rew_bad = ['90 90 90 10', '90 90 90 20', '90 90 90 30', '90 90 90 40', '90 90 90 50', '90 90 90 60',
                         '90 90 10 90', '90 90 20 90', '90 90 30 90', '90 90 40 90', '90 90 50 90', '80 20 80 80',
                         '90 30 90 90', '90 60 90 90', '20 80 80 80', '30 90 90 90', '40 90 90 90', '50 90 90 90',
                         '80 80 80 20', '80 80 80 40', '80 80 80 60', '40 40 40 20', '60 60 60 40', '60 60 60 20',
                         '80 80 20 80', '80 80 40 80', '80 80 60 80', '40 40 20 40', '60 60 40 60', '60 60 20 60',
                         '80 20 80 80', '80 40 80 80', '80 60 80 80', '40 20 40 40', '60 40 60 60', '60 20 60 60',
                         '20 80 80 80', '40 80 80 80', '60 80 80 80', '20 40 40 40', '40 60 60 60', '20 60 60 60']


all_rewards_groups = [constant_group_rew, ascending_rew, descending_rew, one_different_rew_good, one_different_rew_bad]

In [23]:
all_rewards = []
for i in all_rewards_groups:
    for j in i:
        all_rewards.append(j)
        
all_patterns = []
for i in all_patterns_groups:
    for j in i:
        all_patterns.append(j)

In [24]:
new_check_df = pd.DataFrame(columns=all_rewards,index=all_patterns)

In [25]:
output_df = new_check_df.copy()
softmax_df = new_check_df.copy()
output_with_argmax = new_check_df.copy()
output_with_argmin = new_check_df.copy()

In [26]:
tf.reset_default_graph()

In [27]:
for index, row in new_check_df.iterrows():
    raw_index = [int(x)-1 for x in index.split()]    # since actual model gets 0-3
    raw_cols = [r.split() for r in list(row.index)]
#     print(index)
#     print(raw_index)
#     print(raw_cols[0])
    for orig_col in list(row.index):
        # convert current column to array
        cur_col = np.fromstring(orig_col, dtype=int, sep=' ')
        print(new_check_df.loc[index][orig_col])
        print(cur_col)
        row_col_pred, row_col_softmax = make_prediction(raw_index, cur_col)
        output_df.loc[index, orig_col] = row_col_pred+1
        softmax_df.loc[index, orig_col] = np.array2string(np.ravel(row_col_softmax), precision=3, separator=',', suppress_small=True)
        output_with_argmax.loc[index, orig_col] = str((row_col_pred+1)[0]) + " " + np.array2string(np.ravel(row_col_softmax)[row_col_pred], precision=3, separator=',', suppress_small=True)
        output_with_argmin.loc[index, orig_col] = str(np.argmin(np.ravel(row_col_softmax))+1) + " " + np.array2string(np.min(np.ravel(row_col_softmax)), precision=3, separator=',', suppress_small=True)
#     raw_cur_cols = [int(x) for x in raw_cols[0]]
#     make_prediction(raw_index, raw_cur_cols)
    

nan
[10 10 10 10]
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from saved_model/general_full_data_training/model_with_keras_batch2048
predicted class:  [1]
[array([[0.13712242, 0.46373987, 0.26035818, 0.13877949]], dtype=float32)]
nan
[15 15 15 15]
INFO:tensorflow:Restoring parameters from saved_model/general_full_data_training/model_with_keras_batch2048
predicted class:  [1]
[array([[0.28072828, 0.58374995, 0.08670615, 0.04881564]], dtype=float32)]
nan
[20 20 20 20]
INFO:tensorflow:Restoring parameters from saved_model/general_full_data_training/model_with_keras_batch2048
predicted class:  [0]
[array([[0.6834622 , 0

In [28]:
# output_df.to_csv(os.path.join('explain/','output_prediction.csv'))
# softmax_df.to_csv(os.path.join('explain/','output_softmax.csv'))
# output_with_argmax.to_csv(os.path.join('explain/','output_prediction_with_max_softmax.csv'))


#12.2.20
# output_df.to_csv(os.path.join('explain/','output_prediction_V3.csv'))
# softmax_df.to_csv(os.path.join('explain/','output_softmax_V3.csv'))
# output_with_argmax.to_csv(os.path.join('explain/','output_prediction_with_max_softmax_V3.csv'))

# 29.5.20
# output_df.to_csv(os.path.join(saving_dir,'output_prediction_V3.csv'))
# softmax_df.to_csv(os.path.join(saving_dir,'output_softmax_V3.csv'))
# output_with_argmax.to_csv(os.path.join(saving_dir,'output_prediction_with_max_softmax_V3.csv'))

# # 2.6.20
# output_df.to_csv(os.path.join(saving_dir,'output_prediction_V4.csv'))
# softmax_df.to_csv(os.path.join(saving_dir,'output_softmax_V4.csv'))
# output_with_argmax.to_csv(os.path.join(saving_dir,'output_prediction_with_max_softmax_V4.csv'))
# output_with_argmin.to_csv(os.path.join(saving_dir,'argmin_prediction_with_min_softmax_V4.csv'))


# 29.6.20
# output_df.to_csv(os.path.join(saving_dir,'output_prediction_V5.csv'))
# softmax_df.to_csv(os.path.join(saving_dir,'output_softmax_V5.csv'))
# output_with_argmax.to_csv(os.path.join(saving_dir,'output_prediction_with_max_softmax_V5.csv'))
# output_with_argmin.to_csv(os.path.join(saving_dir,'argmin_prediction_with_min_softmax_V5.csv'))

# 14.8.20
output_df.to_csv(os.path.join(saving_dir,'output_prediction_V5.1.csv'))
softmax_df.to_csv(os.path.join(saving_dir,'output_softmax_V5.1.csv'))
output_with_argmax.to_csv(os.path.join(saving_dir,'output_prediction_with_max_softmax_V5.1.csv'))
output_with_argmin.to_csv(os.path.join(saving_dir,'argmin_prediction_with_min_softmax_V5.1.csv'))